In [26]:
import pandas as pd
import requests
import numpy as np

In [2]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

df = pd.DataFrame(documents, columns=['course', 'section', 'question', 'text'])

df.head()

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


In [3]:
docs_example = [
    "January course details, register now",
    "Course prerequisites listed in January catalog",
    "Submit January course homework by end of month",
    "Register for January course, no prerequisites",
    "January course setup: Python and Google Cloud"
]

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')
cv.fit(docs_example)
names = cv.get_feature_names_out()
X = cv.transform(docs_example)
# X.toarray()
df_docs = pd.DataFrame(X.toarray(), columns=names).T


In [5]:
df_docs

,0,1,2,3,4
catalog,0,1,0,0,0
cloud,0,0,0,0,1
course,1,1,1,1,1
details,1,0,0,0,0
end,0,0,1,0,0
google,0,0,0,0,1
homework,0,0,1,0,0
january,1,1,1,1,1
listed,0,1,0,0,0
month,0,0,1,0,0


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(stop_words='english')
X = cv.fit_transform(docs_example)

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs.round(2)


,0,1,2,3,4
catalog,0.00,0.57,0.00,0.00,0.00
cloud,0.00,0.00,0.00,0.00,0.47
course,0.33,0.27,0.23,0.36,0.23
details,0.69,0.00,0.00,0.00,0.00
end,0.00,0.00,0.47,0.00,0.00
google,0.00,0.00,0.00,0.00,0.47
homework,0.00,0.00,0.47,0.00,0.00
january,0.33,0.27,0.23,0.36,0.23
listed,0.00,0.57,0.00,0.00,0.00
month,0.00,0.00,0.47,0.00,0.00


In [7]:
query = "Do I need to know python to sign up for the January course?"
q = cv.transform([query])
query_dict = dict(zip(names, q.toarray()[0]))
query_dict

{'catalog': np.float64(0.0),
 'cloud': np.float64(0.0),
 'course': np.float64(0.39515588491314224),
 'details': np.float64(0.0),
 'end': np.float64(0.0),
 'google': np.float64(0.0),
 'homework': np.float64(0.0),
 'january': np.float64(0.39515588491314224),
 'listed': np.float64(0.0),
 'month': np.float64(0.0),
 'prerequisites': np.float64(0.0),
 'python': np.float64(0.8292789960182417),
 'register': np.float64(0.0),
 'setup': np.float64(0.0),
 'submit': np.float64(0.0)}

In [8]:
doc_dict = dict(zip(names, X.toarray()[1]))
doc_dict



{'catalog': np.float64(0.5675015398728066),
 'cloud': np.float64(0.0),
 'course': np.float64(0.2704175244456293),
 'details': np.float64(0.0),
 'end': np.float64(0.0),
 'google': np.float64(0.0),
 'homework': np.float64(0.0),
 'january': np.float64(0.2704175244456293),
 'listed': np.float64(0.5675015398728066),
 'month': np.float64(0.0),
 'prerequisites': np.float64(0.4578566690891173),
 'python': np.float64(0.0),
 'register': np.float64(0.0),
 'setup': np.float64(0.0),
 'submit': np.float64(0.0)}

In [9]:
from sklearn.metrics.pairwise import cosine_similarity


In [10]:
X.dot(q.T).toarray()


array([[0.25955955],
       [0.21371415],
       [0.17843726],
       [0.28419115],
       [0.57137158]])

In [11]:
fields = ['section', 'question', 'text']
transformers = {}
matrices = {}

for field in fields:
    cv = TfidfVectorizer(stop_words='english', min_df=3)
    X = cv.fit_transform(df[field])

    transformers[field] = cv
    matrices[field] = X

In [14]:
query = "I just singned up. Is it too late to join the course?"
q = transformers['text'].transform([query])
score = cosine_similarity(matrices['text'], q).flatten()
print(score)

[0.3336047  0.         0.         0.1328874  0.         0.
 0.         0.12722114 0.         0.         0.         0.10830554
 0.         0.         0.         0.23530268 0.         0.
 0.04595339 0.         0.         0.         0.22668    0.07952931
 0.         0.         0.         0.1894954  0.08310739 0.
 0.         0.         0.         0.03724346 0.         0.
 0.         0.         0.16484429 0.0231432  0.03424155 0.05174621
 0.         0.03167699 0.         0.         0.         0.
 0.02097473 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.04722243 0.         0.0073737  0.
 0.         0.         0.         0.04161211 0.         0.
 0.         0.         0.         0.03633404 0.         0.
 0.04056748 0.         0.         0.         0.         0.03693003
 0.         0.         0.0321588  0.         0.02220095 0.
 0.         0.03400445 0.         0.         0.         0.
 0.         0.         0

In [17]:
matrices['text'].shape

(948, 2118)

In [18]:
matrices['text']

<948x2118 sparse matrix of type '<class 'numpy.float64'>'
	with 26463 stored elements in Compressed Sparse Row format>

In [20]:
len(df)

948

In [21]:
mask = (df.course == 'data-engineering-zoomcamp').values
score = score * mask
score[:10]

array([0.3336047 , 0.        , 0.        , 0.1328874 , 0.        ,
       0.        , 0.        , 0.12722114, 0.        , 0.        ])

In [28]:
idx = np.argsort(-score)[:10]
df.iloc[idx]

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
15,data-engineering-zoomcamp,General course-related questions,Homework - Are late submissions of homework al...,"No, late submissions are not allowed. But if t..."
22,data-engineering-zoomcamp,General course-related questions,Environment - Do we really have to use GitHub ...,It's up to you which platform and environment ...
27,data-engineering-zoomcamp,General course-related questions,Environment - The GCP and other cloud provider...,You can do most of the course without a cloud....
38,data-engineering-zoomcamp,General course-related questions,Project - What is Project Attemp #1 and Projec...,You will have two attempts for a project. If t...
287,data-engineering-zoomcamp,Module 4: analytics engineering with dbt,CREATE TABLE has columns with duplicate name l...,This error could result if you are using some ...
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
7,data-engineering-zoomcamp,General course-related questions,Course - Can I follow the course after it fini...,"Yes, we will keep all the materials after the ..."
113,data-engineering-zoomcamp,Module 1: Docker and Terraform,"Postgres - ""Column does not exist"" but it actu...","In the join queries, if we mention the column ..."
11,data-engineering-zoomcamp,General course-related questions,Certificate - Can I follow the course in a sel...,"No, you can only get a certificate if you fini..."


In [33]:
boost = {'question': 3.0}

score = np.zeros(len(df))

for f in fields:
    b = boost.get(f, 1.0)
    q = transformers[f].transform([query])
    s = cosine_similarity(matrices[f], q).flatten()
    score = score + b * s


filters = {
    'course': 'data-engineering-zoomcamp'
}

for field, value in filters.items():
    mask = (df[field] == value).values
    score = score * mask

idx = np.argsort(-score)[:10]
results = df.iloc[idx]
results.to_dict(orient='records')

[{'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites'},
 {'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.'},
 {'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.'},
 {'course': 'data-eng

In [34]:
class TextSearch:

    def __init__(self, text_fields):
        self.text_fields = text_fields
        self.matrices = {}
        self.vectorizers = {}

    def fit(self, records, vectorizer_params={}):
        self.df = pd.DataFrame(records)

        for f in self.text_fields:
            cv = TfidfVectorizer(**vectorizer_params)
            X = cv.fit_transform(self.df[f])
            self.matrices[f] = X
            self.vectorizers[f] = cv

    def search(self, query, n_results=10, boost={}, filters={}):
        score = np.zeros(len(self.df))

        for f in self.text_fields:
            b = boost.get(f, 1.0)
            q = self.vectorizers[f].transform([query])
            s = cosine_similarity(self.matrices[f], q).flatten()
            score = score + b * s

        for field, value in filters.items():
            mask = (self.df[field] == value).values
            score = score * mask

        idx = np.argsort(-score)[:n_results]
        results = self.df.iloc[idx]
        return results.to_dict(orient='records')

In [37]:
index = TextSearch(text_fields=['section', 'question', 'text'])
index.fit(documents)
index.search(
    query='I just singned up. Is it too late to join the course?',
    n_results=5,
    boost={'question': 3.0},
    filters={'course': 'data-engineering-zoomcamp'}
)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

In [38]:
from sklearn.decomposition import TruncatedSVD


X = matrices['text']
cv = transformers['text']

svd = TruncatedSVD(n_components=16)
X_emb = svd.fit_transform(X)


query = 'I just singned up. Is it too late to join the course?'

Q = cv.transform([query])
Q_emb = svd.transform(Q)


score = cosine_similarity(X_emb, Q_emb).flatten()
idx = np.argsort(-score)[:10]
list(df.loc[idx].text)


['Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'If you have submitted two projects (and peer-reviewed at least 3 course-mates’ projects for each submission), you will get the certificate for the course. According to the course coordinator, Alexey Grigorev, only two projects are needed to get the course certificate.\n(optional) David Odimegwu',
 'No, it’s not possible. The form is closed after the due date. But don’t worry, homework is not mandatory for finishing the course.',
 'No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp o

In [39]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=16)
X_emb = nmf.fit_transform(X)
Q = cv.transform([query])
Q_emb = nmf.transform(Q)

score = cosine_similarity(X_emb, Q_emb).flatten()
idx = np.argsort(-score)[:10]
list(df.loc[idx].text)

["The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'Please choose the closest one to your answer. Also do not post your answer in the course slack channel.',
 'No, it’s not possible. The form is closed a

In [41]:
import tqdm
import torch
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
model.eval()  # Set the model to evaluation mode if not training

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [42]:
texts = [
    "Yes, we will keep all the materials after the course finishes.",
    "You can follow the course at your own pace after it finishes"
]
encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

In [43]:
encoded_input

{'input_ids': tensor([[  101,  2748,  1010,  2057,  2097,  2562,  2035,  1996,  4475,  2044,
          1996,  2607, 12321,  1012,   102],
        [  101,  2017,  2064,  3582,  1996,  2607,  2012,  2115,  2219,  6393,
          2044,  2009, 12321,   102,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}

In [50]:
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model(**encoded_input)
    hidden_states = outputs.last_hidden_state

In [55]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 1.0103e-01,  1.8106e-02,  1.3034e-01,  ..., -2.9319e-01,
           1.8632e-01,  6.6145e-01],
         [ 1.0608e+00, -1.2425e-01,  1.3701e-01,  ..., -1.6050e-01,
           1.0429e+00,  3.5325e-01],
         [ 1.8022e-01,  7.7587e-02,  3.9414e-01,  ..., -1.3787e-01,
           5.9744e-01,  1.7035e-01],
         ...,
         [ 4.7383e-01, -1.8446e-02,  2.1863e-01,  ..., -1.2873e-03,
          -8.3294e-02, -2.1699e-01],
         [ 6.5164e-01,  1.2163e-01, -2.4941e-01,  ...,  1.5567e-01,
          -5.6319e-01, -4.3100e-01],
         [ 7.1638e-01,  2.1572e-01, -2.8088e-02,  ...,  2.2812e-01,
          -6.7250e-01, -3.2448e-01]],

        [[ 3.1965e-01, -2.4620e-01,  1.9934e-01,  ..., -2.4255e-01,
          -1.0942e-01,  5.8847e-01],
         [-6.9823e-01, -7.5619e-01,  1.0645e-01,  ..., -1.1348e-01,
           4.5499e-01,  4.0241e-01],
         [ 4.1643e-01, -4.7885e-01,  3.2889e-01,  ..., -5.1462e-01,
           6.

In [60]:
hidden_states.shape
# print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
# layer_i = 0

# print ("Number of batches:", len(hidden_states[layer_i]))
# batch_i = 0

# print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
# token_i = 0

# print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

torch.Size([2, 15, 768])

In [57]:
sentence_embeddings = hidden_states.mean(dim=0)
sentence_embeddings.shape

torch.Size([2, 768])

In [47]:
sentence_embeddings.numpy()


array([[ 0.35999218, -0.1607233 ,  0.3545232 , ...,  0.04289275,
         0.0348231 , -0.03822241],
       [ 0.17849962, -0.5000252 ,  0.25277552, ..., -0.11413086,
        -0.33608466,  0.41095114]], dtype=float32)

In [48]:
def make_batches(seq, n):
    result = []
    for i in range(0, len(seq), n):
        batch = seq[i:i+n]
        result.append(batch)
    return result


from tqdm.auto import tqdm

def compute_embeddings(texts, batch_size=8):
    text_batches = make_batches(texts, 8)
    
    all_embeddings = []
    
    for batch in tqdm(text_batches):
        encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
    
        with torch.no_grad():
            outputs = model(**encoded_input)
            hidden_states = outputs.last_hidden_state
            
            batch_embeddings = hidden_states.mean(dim=1)
            batch_embeddings_np = batch_embeddings.cpu().numpy()
            all_embeddings.append(batch_embeddings_np)
    
    final_embeddings = np.vstack(all_embeddings)
    return final_embeddings


embeddings = {}


for f in fields:
    print(f'computing embeddings for {f}...')
    embeddings[f] = compute_embeddings(df[f].tolist())

computing embeddings for section...


100%|███████████████████████████████████████████████████████████████████| 119/119 [00:04<00:00, 29.34it/s]


computing embeddings for question...


100%|███████████████████████████████████████████████████████████████████| 119/119 [00:08<00:00, 14.61it/s]


computing embeddings for text...


100%|███████████████████████████████████████████████████████████████████| 119/119 [00:58<00:00,  2.05it/s]


In [49]:
embeddings

{'section': array([[ 0.3774863 , -0.16826616, -0.71794647, ...,  0.32759294,
         -0.12342925,  0.18709973],
        [ 0.37748614, -0.16826595, -0.7179467 , ...,  0.32759282,
         -0.12342954,  0.18709978],
        [ 0.37748647, -0.16826598, -0.71794635, ...,  0.32759333,
         -0.12342899,  0.18709973],
        ...,
        [-0.17838065, -0.00579748, -0.19219293, ..., -0.09306467,
          0.06128895, -0.07417862],
        [-0.17838039, -0.00579755, -0.19219235, ..., -0.0930646 ,
          0.06128889, -0.07417917],
        [-0.17838061, -0.00579736, -0.1921928 , ..., -0.09306439,
          0.06128892, -0.0741794 ]], dtype=float32),
 'question': array([[-6.5593086e-02, -3.2150456e-01,  5.1302648e-01, ...,
         -8.1999421e-02, -1.2137162e-01,  3.8852159e-02],
        [ 2.2855531e-01,  4.2073984e-02,  2.0274167e-01, ...,
         -8.8864855e-02,  4.9351156e-04,  8.1972714e-04],
        [ 3.4746408e-02, -2.7245417e-01,  2.2815709e-01, ...,
          5.9614759e-02, -1.28633